In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

In [2]:
df=pd.read_csv('heart_disease_uci.csv')
df.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [3]:
df.isnull().sum()

id            0
age           0
sex           0
dataset       0
cp            0
trestbps     59
chol         30
fbs          90
restecg       2
thalch       55
exang        55
oldpeak      62
slope       309
ca          611
thal        486
num           0
dtype: int64

In [4]:
df_clean = df.drop(columns=['ca', 'thal', 'slope'])

In [18]:
imputer =SimpleImputer(strategy='median')
df_clean[['trestbps','chol','thalch','oldpeak']]=imputer.fit_transform(df_clean[['trestbps', 'chol', 'thalch', 'oldpeak']])

In [19]:
df_clean['sex'] = df_clean['sex'].map({'Male': 1, 'Female': 0})
df_clean['exang'] = df_clean['exang'].map({'True': 1, 'False': 0})
df_clean['fbs'] = df_clean['fbs'].map({'True': 1, 'False': 0})

In [20]:
df_cleaned = pd.get_dummies(df_clean, columns=['cp', 'restecg', 'dataset'], drop_first=True)


In [21]:
df_clean.isnull().sum()

id            0
age           0
sex         920
dataset       0
cp            0
trestbps      0
chol          0
fbs         920
restecg       2
thalch        0
exang       920
oldpeak       0
num           0
dtype: int64

In [15]:
y = df_clean['num'].apply(lambda x:1 if x>0 else 0)
X=df_clean.drop(columns=['id','num'])

In [16]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [26]:

lr = LogisticRegression(random_state=42)

# Fit the model to your data
lr.fit(X_train, y_train)


ValueError: could not convert string to float: 'VA Long Beach'

In [28]:
# Check the data types of each column
print(X_train.dtypes)


age           int64
sex         float64
dataset      object
cp           object
trestbps    float64
chol        float64
fbs         float64
restecg      object
thalch      float64
exang       float64
oldpeak     float64
dtype: object


In [30]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Convert each string column
for column in X_train.select_dtypes(include=[object]).columns:
    X_train[column] = label_encoder.fit_transform(X_train[column])


In [31]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse_output=False, drop='first')

# Convert categorical columns to one-hot encoded format
X_train_categorical = X_train.select_dtypes(include=[object])
X_train_encoded = one_hot_encoder.fit_transform(X_train_categorical)

# Convert the encoded features to a DataFrame
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=one_hot_encoder.get_feature_names_out())

# Drop the original categorical columns from X_train
X_train = X_train.drop(columns=X_train_categorical.columns)

# Concatenate the encoded features with the remaining numeric columns
X_train = pd.concat([X_train, X_train_encoded_df], axis=1)


TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'